In [1]:
import pandas as pd


from constants import okr_excel_path, researcher_info_excel_path, salesperson_info_excel_path, output_excel_path
from utils import get_df_roadshow, get_sales_name_list, get_sales_name_list, write_dfs_to_excel
from utils import print_na_rate, get_df_researcher

In [2]:
df_okr = pd.read_excel(okr_excel_path, sheet_name='路演')
df_researcher_info = pd.read_excel(researcher_info_excel_path, usecols='A:C')
df_salespeople_info =  pd.read_excel(salesperson_info_excel_path)

In [3]:
sales_name_list = get_sales_name_list(df_salespeople_info)

In [4]:
df_roadshow, df_special = get_df_roadshow(df_okr, sales_name_list)
df_researcher = get_df_researcher(df_roadshow)

In [5]:
df_researcher

,研究员,所属团队,总路演次数,客户内外部策略会,晨会接入,路演-专家路演,路演-反路演(董事长/总经理),路演-反路演(董秘/IR),路演-线上路演,路演-线下路演,路演-餐叙路演,54A路演次数,上海,北京,广深
111,陈良栋,传媒,23,1.0,0.0,2.0,0.0,0.0,4.0,11.0,5.0,11,21.0,1.0,1.0
31,尚硕,公用事业,19,0.0,0.0,0.0,0.0,1.0,9.0,9.0,0.0,12,2.0,3.0,13.0
110,赵国利,公用事业,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0.0,0.0,0.0
13,严家源,公用事业,72,0.0,0.0,1.0,0.0,1.0,44.0,21.0,5.0,57,31.0,28.0,12.0
39,孔厚融,军工,10,0.0,0.0,0.0,0.0,1.0,9.0,0.0,0.0,2,5.0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,孙冉,食品饮料,22,0.0,0.0,3.0,3.0,12.0,3.0,0.0,1.0,17,12.0,7.0,3.0
20,王言海,食品饮料,19,1.0,0.0,0.0,0.0,0.0,5.0,12.0,1.0,13,16.0,2.0,1.0
88,李啸,食品饮料,20,0.0,0.0,0.0,0.0,0.0,7.0,10.0,3.0,12,0.0,15.0,5.0
105,杜山,食品饮料,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,0.0,1.0,0.0


In [6]:
def get_unique_teams_per_roadshow(df):
    """
    Get the number of unique teams per roadshow
    """
    unique_teams_per_roadshow = df.groupby('序号')['所属团队'].nunique().reset_index(name='unique_teams_count')
    return unique_teams_per_roadshow

In [15]:
def get_df_unique(df_roadshow):
    unique_teams_per_roadshow = df_roadshow.groupby('序号')['所属团队'].nunique().reset_index(name='unique_teams_count')
    merged_df = df_roadshow.merge(unique_teams_per_roadshow, on='序号', how='left')
    multi_team_rows = merged_df[merged_df['unique_teams_count'] > 1].drop_duplicates(subset=['序号', '所属团队'])
    single_team_rows = merged_df[merged_df['unique_teams_count'] == 1].drop_duplicates(subset=['序号'])
    filtered_df = pd.concat([multi_team_rows, single_team_rows])
    return filtered_df


In [16]:
filtered_df = get_df_unique(df_roadshow)

In [17]:
filtered_df

,序号,服务事项,客户机构,客户分级,客户区域,所属团队,研究员,unique_teams_count
0,1,路演-线上路演,安联保险资产管理有限公司,A,北京,计算机,郭新宇,1
1,2,路演-餐叙路演,安信基金,3A,广深,计算机,刘雄,1
2,3,路演-线下路演,安信基金,3A,广深,医药,宋丽莹,1
3,4,路演-线下路演,安信基金,3A,广深,能源开采,周泰,1
4,5,路演-反路演(董秘/IR),安信基金,3A,广深,电新,邓永康,1
...,...,...,...,...,...,...,...,...
2697,2724,路演-线上路演,悟空投资,A,上海,食品饮料,张玲玉,1
2698,2725,路演-专家路演,悟空投资,A,上海,传媒,陈良栋,1
2699,2726,路演-专家路演,悟空投资,A,上海,传媒,陈良栋,1
2700,2727,路演-线上路演,悟空投资,A,上海,食品饮料,胡慧铭,1


In [7]:
# wrap the following codes to a function
def get_df_team(df_roadshow):
    filtered_df = get_df_unique(df_roadshow)

    total_roadshows_per_team = filtered_df.groupby('所属团队')['序号'].count().reset_index(name='总路演次数')
    service_counts_per_team = filtered_df.groupby(['所属团队', '服务事项']).size().reset_index(name='count')
    pivot_service_counts = service_counts_per_team.pivot(index='所属团队', columns='服务事项', values='count').fillna(0).astype(int).reset_index()
    result_df = total_roadshows_per_team.merge(pivot_service_counts, on='所属团队', how='left')

    return result_df

df_team = get_df_team(df_roadshow)


In [8]:
unique_teams_per_roadshow = get_unique_teams_per_roadshow(df_roadshow)

# Merge the unique_teams_per_roadshow DataFrame with the original DataFrame
merged_df = df_roadshow.merge(unique_teams_per_roadshow, on='序号', how='left')

# Filter out the rows where there are multiple teams per roadshow and drop duplicates
multi_team_rows = merged_df[merged_df['unique_teams_count'] > 1].drop_duplicates(subset=['序号', '所属团队'])

# Filter out the rows where there is only one team per roadshow and drop duplicates based on roadshow index
single_team_rows = merged_df[merged_df['unique_teams_count'] == 1].drop_duplicates(subset=['序号'])

# Concatenate the two DataFrames
filtered_df = pd.concat([multi_team_rows, single_team_rows])

# Now proceed with the original analysis

# count the number of each type of roadshow for each team
total_roadshows_per_team = filtered_df.groupby('所属团队')['序号'].count().reset_index(name='总路演次数')
service_counts_per_team = filtered_df.groupby(['所属团队', '服务事项']).size().reset_index(name='count')
pivot_service_counts = service_counts_per_team.pivot(index='所属团队', columns='服务事项', values='count').fillna(0).astype(int).reset_index()
result_df = total_roadshows_per_team.merge(pivot_service_counts, on='所属团队', how='left')

# calculate 5A and A4 roadshow counts for each team



In [9]:
unique_teams_per_roadshow

,序号,unique_teams_count
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
2539,2724,1
2540,2725,1
2541,2726,1
2542,2727,1


In [10]:
merged_df[merged_df.unique_teams_count > 1]

,序号,服务事项,客户机构,客户分级,客户区域,所属团队,研究员,unique_teams_count


In [13]:
result_df

,所属团队,总路演次数,客户内外部策略会,晨会接入,路演-专家路演,路演-反路演(董事长/总经理),路演-反路演(董秘/IR),路演-线上路演,路演-线下路演,路演-餐叙路演
0,传媒,23,1,0,2,0,0,4,11,5
1,公用事业,92,0,0,1,0,2,54,30,5
2,军工,50,0,0,1,0,5,30,11,3
3,农业,16,0,0,0,0,0,10,3,3
4,化工,62,0,0,1,1,20,32,6,2
5,医药,154,0,1,9,7,8,37,75,17
6,商社,149,0,0,8,0,17,52,56,16
7,固收,53,0,0,1,0,1,10,30,11
8,地产,48,0,0,3,0,0,14,25,6
9,宏观,127,1,6,0,0,0,30,73,17


In [11]:
# # Drop the region column
# df_without_region = df_researcher.drop(columns=['参与地区', '研究员'])

# # Group by and calculate sum, median, and mode
# sum_df = df_without_region.groupby('所属团队').sum().reset_index()
# median_df = df_without_region.groupby('所属团队').median().reset_index()
# mode_df = df_without_region.groupby('所属团队').agg(lambda x: x.mode().iloc[0]).reset_index()

# # Merging dataframes
# df_team = sum_df.merge(median_df, on='所属团队', suffixes=('_sum', '_median'))
# for col in df_team.columns:
#     if not col.endswith(('sum', 'median', '所属团队')):
#         df_team[col + '_mode'] = mode_df[col]

# # Rename the '所属团队' column
# df_team.rename(columns={'所属团队': '行业'}, inplace=True)




In [12]:
dfs_dict = {
            'roadshow': df_roadshow,
            'special': df_special,
            'researcher': df_researcher,
            'team': df_team
            }


write_dfs_to_excel(dfs_dict, okr_excel_path)